In [16]:
# notebook to illustrate co-occurrence graphs

In [17]:
# following only used for development, reloads the modules with any code changes
%load_ext autoreload
%autoreload 2

# inline matplotlib charts
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
# import our text mining toolkit
import text_mining_toolkit as tmt

In [19]:
cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/simple_test/txt/", text_filename_pattern="??.txt")
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/recipes/txt/", text_filename_pattern="??.txt")
#cr = tmt.corpus_reader.CorpusReader(directory_of_files="data_sets/mystery_corpus_01/txt/", text_filename_pattern="??.txt")
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/iraq_inquiry/txt/", text_filename_pattern="the-report*.txt")
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/clinton_emails/txt/", text_filename_pattern="C0*.txt")
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/shakespeare_macbeth/txt/", text_filename_pattern="macbeth_act_0?_scene_0?.txt")

content_directory =  data_sets/simple_test/txt/
text_filename_pattern =  ??.txt
self.documents populated =  5


In [20]:
# load previously calculated relevance scores
words_by_relevance = tmt.index_relevance.get_words_by_relevance(cr.content_directory)
# top most relevant
top_words_by_relevance = words_by_relevance[:100]

In [21]:
# first clear index
tmt.index_cooccurrence.delete_matrix(cr.content_directory)

# for all documents in corpus
for document_name in cr.get_documents():
    print("processing ", document_name)

    # get document text
    document_text = cr.get_text_by_document(document_name)

    # simplify whitespace (remove newlines)
    b = tmt.text_processing.simplify_whitespace(document_text)

    # only keep alphanumeric characters, removes punctuation
    c = tmt.text_processing.keep_only_alphanumeric(b)

    # make lowercase
    d = tmt.text_processing.to_lowercase(c)

    # split into words list
    dl = tmt.text_processing.split_text_into_words(d)
    # build n-grams
    #gl = tmt.word_processing.build_ngrams_from_words(dl,2)

    # remove stop words
    el = tmt.word_processing.remove_stop_words(dl, "./stopwords/minimal-stop.txt")
    
    # remove all except those in the top most relevant list
    fl = [word for word in el if word in top_words_by_relevance]
    
    # update index
    tmt.index_cooccurrence.create_cooccurrence_matrix_for_document(cr.content_directory, document_name, fl, window=4)
    pass


removed co-occurrence matrix file:  data_sets/simple_test/txt/matrix.cooccurrence
processing  01.txt
processing  02.txt
processing  03.txt
processing  04.txt
processing  05.txt


In [22]:
# merge document indices into a corpus index
tmt.index_cooccurrence.merge_cooccurrence_matrices_for_corpus(cr.content_directory)

saving corpus co-occurrence matrix ...  data_sets/simple_test/txt/matrix.cooccurrence


In [23]:
tmt.index_cooccurrence.print_matrix(cr.content_directory)

cooccurrence_matrix_file  data_sets/simple_test/txt/matrix.cooccurrence
           apple    banana      kiwi    orange
apple   2.657627  0.000000  0.000000  2.287997
banana  0.000000  0.000000  0.000000  2.287997
orange  0.000000  0.939413  0.939413  4.374090


In [24]:
# query the co-occurrence value for a given pair of words
tmt.index_cooccurrence.query_cooccurance_matrix(cr.content_directory, "apple", "kiwi")

0.0

In [27]:
# get the most likely next word by co-occurrence value for given word
tmt.index_cooccurrence.most_likely_next(cr.content_directory, "apple")

'apple'

In [28]:
#generate sentences based on co-occurence
sentence_length = 7

#initial words 
#for w in ["the", "once", "then", "olive", "tomato", "cut", "cook", "little", "boiled"]:
for w in ["apple"]:
    sentence = w
    for i in range(sentence_length):
        w = tmt.index_cooccurrence.most_likely_next(cr.content_directory, w)
        sentence += (" " + w)
        pass
    print(sentence)
    pass


apple apple apple apple apple apple apple apple


In [29]:
words_by_co_occurrence = tmt.index_cooccurrence.get_word_pairs_by_cooccurrence(cr.content_directory)

In [30]:
# 20 most common
words_by_co_occurrence[:20]

,word1,word2,weight
0,orange,orange,1.000000
1,apple,apple,0.607584
2,banana,orange,0.523079
3,apple,orange,0.523079
4,orange,kiwi,0.214768
5,orange,banana,0.214768


In [31]:
# 20 least common
words_by_co_occurrence[-20:]

,word1,word2,weight
0,orange,orange,1.000000
1,apple,apple,0.607584
2,banana,orange,0.523079
3,apple,orange,0.523079
4,orange,kiwi,0.214768
5,orange,banana,0.214768


In [32]:
# visualise co-occurrence of words
tmt.visualisation.plot_force_directed_graph(words_by_co_occurrence[:500])

force-directed graph

In [33]:
# visualise co-occurrence > 1
w = words_by_co_occurrence[words_by_co_occurrence['weight']>0.2]
tmt.visualisation.plot_force_directed_graph(w)

force-directed graph